In [ ]:
import os
import time
from pathlib import Path
import pandas as pd
import QuantLib as ql
import numpy as np
from datetime import datetime
from model_settings import ms
from itertools import product
from joblib import Parallel, delayed

In [ ]:
from model_settings import asian_option_pricer
aop = asian_option_pricer.asian_option_pricer()

# functions in package

In [ ]:
def asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,past_fixings,kappa,theta,rho,eta,v0,calculation_datetime):
    s = float(s)
    k = float(k)
    r = float(r)
    g = float(g)
    rng = "pseudorandom" # could use "lowdiscrepancy"
    numPaths = 100000
    
    if w == 'call':
        option_type = ql.Option.Call 
    elif w == 'put':
        option_type = ql.Option.Put
    t = n_fixings*fixing_frequency
    
    calculation_date = ql.Date(calculation_datetime.day,calculation_datetime.month,calculation_datetime.year)
    
    periods = np.arange(fixing_frequency,t+1,fixing_frequency).astype(int)
    
    fixing_dates = [calculation_date + ql.Period(int(p),ql.Days) for p in periods]
    expiration_date = calculation_date + ql.Period(int(t),ql.Days)
    
    riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, float(r), ql.Actual365Fixed()))
    dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, float(g), ql.Actual365Fixed()))
    
    hestonProcess = ql.HestonProcess(riskFreeTS, dividendTS, ql.QuoteHandle(ql.SimpleQuote(s)), v0, kappa, theta, eta, rho)
    hestonModel = ql.HestonModel(hestonProcess)
    vanillaPayoff = ql.PlainVanillaPayoff(option_type, float(k))
    europeanExercise = ql.EuropeanExercise(expiration_date)
    
    if averaging_type == 'geometric':
        geometric_engine = ql.MCDiscreteGeometricAPHestonEngine(hestonProcess, rng, requiredSamples=numPaths,seed=123)
        geometricAverage = ql.Average().Geometric
        geometricRunningAccumulator = 1.0
        discreteGeometricAsianOption = ql.DiscreteAveragingAsianOption(
        	geometricAverage, geometricRunningAccumulator, past_fixings,
            fixing_dates, vanillaPayoff, europeanExercise)
        discreteGeometricAsianOption.setPricingEngine(geometric_engine)
        geometric_price = float(discreteGeometricAsianOption.NPV())
        return geometric_price
        
    elif averaging_type == 'arithmetic':
        arithmetic_engine = ql.MCDiscreteArithmeticAPHestonEngine(hestonProcess, rng, requiredSamples=numPaths)
        arithmeticAverage = ql.Average().Arithmetic
        arithmeticRunningAccumulator = 0.0
        discreteArithmeticAsianOption = ql.DiscreteAveragingAsianOption(
            arithmeticAverage, arithmeticRunningAccumulator, past_fixings, 
            fixing_dates, vanillaPayoff, europeanExercise)
        discreteArithmeticAsianOption.setPricingEngine(arithmetic_engine)
        arithmetic_price = float(discreteArithmeticAsianOption.NPV())
        return arithmetic_price
    else:
        print("invalid Asian option averaging type out of 'arithmetic' and geometric'")
        pass

In [ ]:
def row_asian_option_price(row):
    return  asian_option_price(
        row['spot_price'],
        row['strike_price'],
        row['risk_free_rate'],
        row['dividend_rate'],
        row['w'],
        row['averaging_type'],
        row['fixing_frequency'],
        row['n_fixings'],
        row['past_fixings'],
        row['kappa'],
        row['theta'],
        row['rho'],
        row['eta'],
        row['v0'],
        row['calculation_date']
    )

In [ ]:
def df_asian_option_price(df):
    prices = Parallel(n_jobs=-1)(delayed(row_asian_option_price)(row) for _, row in df.iterrows())
    return prices

# example usage

In [ ]:
calculation_datetime = datetime.today()
"""
365.41	548	 0.04	0.0	put	geometric	90	10	0	0.412367	0.17771	-0.582856	0.785592	0.08079	2024-10-18 13:38:09.345440	900	147.330432	156.641565
"""
r = 0.04
g = 0.0

w = 'put'

past_fixings = 0
s,k = 365.41, 548
# s = 365.41
# k = 370.00
kappa = 0.412367
theta = 0.17771
rho = -0.582856
eta = 0.785592
v0 = 0.08079
n_fixings = 10
fixing_frequency = 90

averaging_type = 'geometric'

t = n_fixings*fixing_frequency


price = asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,past_fixings,kappa,theta,rho,eta,v0,calculation_datetime)
package_price = aop.asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,past_fixings,kappa,theta,rho,eta,v0,calculation_datetime)
vanilla = ms.ql_heston_price(s,k,t,r,g,w,kappa,theta,rho,eta,v0,datetime.today())

print(f"asian: {price}  {package_price}\nvanilla: {vanilla}")

In [ ]:
fixing_frequencies = [
    1,7,
    30,
    90
]
n_fixings = [
    5,
    10
]
K = np.unique(np.linspace(s*0.5,s*1.5,50).astype(int))

W = [
    'call',
    'put'
]
types = [
    'arithmetic',
    'geometric'
]
past_fixings = [0]
features = pd.DataFrame(
    product(
        [s],
        K,
        [r],
        [g],
        W,
        types,
        fixing_frequencies,
        n_fixings,
        past_fixings,
        [kappa],
        [theta],
        [rho],
        [eta],
        [v0],
        [calculation_datetime]
    ),
    columns = [
        'spot_price','strike_price','risk_free_rate','dividend_rate','w',
        'averaging_type','fixing_frequency','n_fixings','past_fixings',
        'kappa','theta','rho','eta','v0','calculation_date'
    ]
)
features['days_to_maturity'] = features['n_fixings']*features['fixing_frequency']

In [ ]:
# start = time.time()
# features['asian_price'] = features.apply(asian_option_pricer.row_asian_option_price,axis=1)
# end = time.time()
# print("pandas apply cpu:")
# print(end-start)
# print()

start= time.time()
features['asian_price'] = aop.df_asian_option_price(features)
end = time.time()
print("parallelized cpu:")
print(end-start)

In [ ]:
features